In [1]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saumya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import os
import pandas as pd
from ast import literal_eval
from utils import helpersorigin
import networkx as nx
def GetOriginPublication(df):    
    return df.loc[df['year'].idxmin()]['id']

#helpersorigin.GetPublications('lstm') #uncomment and run for new concept

#L0
#find original publication
directory_path=os.path.expanduser("~")
df_all=pd.read_csv(directory_path+'\OriginAndEvolutionofConcept\data\outputs\df_concept.csv')
df_originid=GetOriginPublication(df_all)
df_origin=df_all.loc[(df_all['id'] == df_originid)]

#L1
#process outcitations of origin
df_origin['outCitations']=df_origin['outCitations'].apply(lambda x:literal_eval(x))

#find similarities in abstract L0L1

#helpersorigin.GetDetailsofReferences(df_origin['outCitations'][df_origin.index[0]]) #uncomment for new concept
df_oo_l1l2=pd.read_csv(directory_path+'\OriginAndEvolutionofConcept\data\outputs\df_reference_details.csv')
df_oo_l0l1l2=df_oo_l1l2.append(df_origin)
tfidf = vectorizer.fit_transform(df_oo_l0l1l2['paperAbstract'])
similarities=((tfidf * tfidf.T).A)[len(df_oo_l0l1l2.index)-1]
df_oo_l0l1l2['similarity']=similarities

#process nodes for gephi input
df_nodes=df_oo_l0l1l2[['id','similarity']]
df_nodes['label']=df_oo_l0l1l2['title']
df_nodes.to_csv(directory_path+'\OriginAndEvolutionofConcept\data\outputs\l0l1sim_nodes.csv',index=False)

#process edges for gephi input
df_edges=df_oo_l0l1l2[['id','similarity']]
df_edges['source']=df_originid
df_edges.columns=['target','similarity','source']
df_edges.to_csv(directory_path+'\OriginAndEvolutionofConcept\data\outputs\l0l1sim_edges.csv',index=False)

C:\Users\saumya\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\saumya\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\saumya\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind